### Oklahoma purchase credit card transactions

The Office of Management and Enterprise Services in the State of Oklahoma has made its [purchase credit card transactions](https://catalog.data.gov/dataset/purchase-card-pcard-fiscal-year-2014) available. This dataset contains information on purchases made through the purchase card programs administered by the state and higher education institutions. 

### Prepare the dataset 

When you model, you will ask youself the goal of the modeling purpose. Our purpose is to detect suspicious behaviors for an individual agency for a particular merchant category. We therefore propose two ways to organize the modeling datasets:
1. Each row in the modeling dataset is a record of an agency name. 
2. Each row in the modeling dataset is a record of a merchant category of an agency name. 

If the dataset is (1), the variables shall detail the statistics for each merchant category. For example, the variables can be "Merchant1_1wk_dol" to "Merchant100_1wk_dol" for the dollar amount in 1 week for Merchant Category 1 to 100. Similarly, other merchant-category variables such as XX weeks or number of transactions can be created.  

If the dataset is (2), the variables can be the Recency-Frequency_Monetary (RFM) variables for each merchant category of an agency name. In the following program, we will proceed with (2). You are encouraged to test (1) as well.

### Some considerations on the variables:
Obviously the spending amounts will vary widely by agency name and merchant category. In many unsupervised methods such as cluster analysis, data points of similar magnitudes tend to be clustered together. For example, the following table shows the average amounts for Agency A and B for their merchant categories. A cluster analysis may group Record 1, 2, 3 together, and leave Record 4 alone. Is this result reasonable?    

<table>
  <tr>      
    <th>#</th>
    <th>Agency Name</th>
    <th>Category</th>
    <th>Month 1</th>
    <th>Month 2</th>
    <th>Month 3</th>
  </tr>
  <tr>
    <td>1</td>
    <td>A</td>
    <td>Stationary</td>
    <td>98</td>
    <td>100</td>
    <td>102</td>
  </tr>
  <tr>
    <td>2</td>
    <td>A</td>
    <td>Gas</td>
    <td>196</td>
    <td>200</td>
    <td>204</td>
  </tr>
  <tr>
    <td>3</td>
    <td>B</td>
    <td>Pets</td>
    <td>96</td>
    <td>100</td>
    <td>104</td>
  </tr>
  <tr>
    <td>4</td>
    <td>B</td>
    <td>Supply</td>
    <td>1050</td>
    <td>1000</td>
    <td>950</td>
  </tr>
</table>

</body>
</html>

In most cases we need to derive variables to avoid the unwanted situations. What if we get the average value for Month 1 to 3 for each agency and merchant category, and then divide the amount the average value to get ratios as the following? The clustering result will be different.

<table>
  <tr>      
    <th>#</th>
    <th>Agency Name</th>
    <th>Category</th>
    <th>Month 1</th>
    <th>Month 2</th>
    <th>Month 3</th>
  </tr>
  <tr>
    <td>1</td>
    <td>A</td>
    <td>Stationary</td>
    <td>0.98</td>
    <td>1.00</td>
    <td>1.02</td>
  </tr>
  <tr>
    <td>2</td>
    <td>A</td>
    <td>Gas</td>
    <td>0.98</td>
    <td>1.00</td>
    <td>1.02</td>
  </tr>
  <tr>
    <td>3</td>
    <td>B</td>
    <td>Pets</td>
    <td>0.96</td>
    <td>1.00</td>
    <td>1.02</td>
  </tr>
  <tr>
    <td>4</td>
    <td>B</td>
    <td>Supply</td>
    <td>1.05</td>
    <td>1.00</td>
    <td>0.95</td>
  </tr>
</table>

</body>
</html>

In [2]:
library(dplyr)
#library(DataExplorer)
#library(xda)
library(ggplot2)
library(plotly)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘plotly’

The following object is masked from ‘package:ggplot2’:

    last_plot

The following object is masked from ‘package:stats’:

    filter

The following object is masked from ‘package:graphics’:

    layout



In [1]:
ccard <- read.csv("/Users/chriskuo/Downloads/purchase_credit_card.csv")

In [3]:
dim(ccard)
summary(ccard)
colnames(ccard)

[1] 442458     11

   Year.Month     Agency.Number  
 Min.   :201307   Min.   : 1000  
 1st Qu.:201309   1st Qu.: 1000  
 Median :201401   Median :47700  
 Mean   :201357   Mean   :42786  
 3rd Qu.:201404   3rd Qu.:76000  
 Max.   :201406   Max.   :98000  
                                 
                                Agency.Name    
 OKLAHOMA STATE UNIVERSITY            :115995  
 UNIVERSITY OF OKLAHOMA               : 76143  
 UNIV. OF OKLA. HEALTH SCIENCES CENTER: 58247  
 DEPARTMENT OF CORRECTIONS            : 22322  
 DEPARTMENT OF TOURISM AND RECREATION : 17232  
 DEPARTMENT OF TRANSPORTATION         : 15689  
 (Other)                              :136830  
               Cardholder.Last.Name Cardholder.First.Initial
 JOURNEY HOUSE TRAVEL INC: 10137    J      : 55031          
 UNIVERSITY AMERICAN     :  7219    G      : 42251          
 JOURNEY HOUSE TRAVEL    :  4693    D      : 38120          
 Heusel                  :  4212    M      : 35352          
 Hines                   :  3423    S  

[1] "Year.Month"                   "Agency.Number"               
 [3] "Agency.Name"                  "Cardholder.Last.Name"        
 [5] "Cardholder.First.Initial"     "Description"                 
 [7] "Amount"                       "Vendor"                      
 [9] "Transaction.Date"             "Posted.Date"                 
[11] "Merchant.Category.Code..MCC."

In [4]:
# Count of agencies
# Spent by agency
# Count by merchant.Category.Code
colnames(ccard)<-c('Year_Month', 'Agency_Number', 'Agency_Name', 'Cardholder_Last_Name',
      'Cardholder_First_Initial', 'Description', 'Amount', 'Vendor', 'Transaction_Date',
      'Posted_Date', 'Merchant_Category')

In [5]:
nrow(ccard)
head(ccard)
table(ccard$Year_Month)

[1] 442458

Year_Month,Agency_Number,Agency_Name,Cardholder_Last_Name,Cardholder_First_Initial,Description,Amount,Vendor,Transaction_Date,Posted_Date,Merchant_Category
201307,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,GENERAL PURCHASE,890.00,NACAS,07/30/2013 12:00:00 AM,07/31/2013 12:00:00 AM,CHARITABLE AND SOCIAL SERVICE ORGANIZATIONS
201307,1000,OKLAHOMA STATE UNIVERSITY,Mason,C,ROOM CHARGES,368.96,SHERATON HOTEL,07/30/2013 12:00:00 AM,07/31/2013 12:00:00 AM,SHERATON
201307,1000,OKLAHOMA STATE UNIVERSITY,Massey,J,GENERAL PURCHASE,165.82,SEARS.COM 9300,07/29/2013 12:00:00 AM,07/31/2013 12:00:00 AM,DIRCT MARKETING/DIRCT MARKETERS--NOT ELSEWHERE CLASSIFIED
201307,1000,OKLAHOMA STATE UNIVERSITY,Massey,T,GENERAL PURCHASE,96.39,WAL-MART #0137,07/30/2013 12:00:00 AM,07/31/2013 12:00:00 AM,"GROCERY STORES,AND SUPERMARKETS"
201307,1000,OKLAHOMA STATE UNIVERSITY,Mauro-Herrera,M,HAMMERMILL COPY PLUS COPY EA,125.96,STAPLES DIRECT,07/30/2013 12:00:00 AM,07/31/2013 12:00:00 AM,"STATIONERY, OFFICE SUPPLIES, PRINTING AND WRITING PAPER"
201307,1000,OKLAHOMA STATE UNIVERSITY,Mauro-Herrera,M,GENERAL PURCHASE,394.28,KYOCERA DOCUMENT SOLUTION,07/29/2013 12:00:00 AM,07/31/2013 12:00:00 AM,"OFFICE, PHOTOGRAPHIC, PHOTOCOPY, AND MICROFILM EQUIPMENT"



201307 201308 201309 201310 201311 201312 201401 201402 201403 201404 201405 
 37635  39314  38762  40266  34275  26969  37230  35831  38188  39249  36784 
201406 
 37955 

### Data preparation

In [10]:
# Calculate the average amount by agency_name and erchant category
avg_agency <- ccard %>% group_by(Agency_Name, Merchant_Category) %>%
       summarise( mean_category_amount = mean(Amount),
                  mean_count_trans =n()
                )
head(avg_agency)

# Append the average statistics back to the data to derive the ratios.
# Select the most recent 4 transactions 
per_agency_category <- ccard %>% group_by(Agency_Name, Merchant_Category, Year_Month) %>%
       summarise( max_amount = max(Amount),
                  mean_amount = mean(Amount),
                  count_trans =n()
                ) %>%
       left_join(avg_agency, by=c('Agency_Name','Merchant_Category')) %>%
       mutate( max_amount_ratio = max_amount / mean_category_amount,
               mean_amount_ratio = mean_amount / mean_category_amount,
               mean_count_ratio  = count_trans / mean_count_trans
       ) %>% select(-mean_category_amount,-mean_count_trans, -max_amount, -mean_amount, -count_trans) %>%
        top_n(-4)  # Use top_n(xx) to select the top xx rows, and top_n(-xx) for the bottom xx rows

per_agency_category[1:100,]

Agency_Name,Merchant_Category,mean_category_amount,mean_count_trans
`DEPARTMENT OF EDUCATION,ADVERTISING SERVICES,508.48600,5
`DEPARTMENT OF EDUCATION,AMERICAN AIRLINES,492.98809,89
`DEPARTMENT OF EDUCATION,BEST WESTERN,94.81143,7
`DEPARTMENT OF EDUCATION,BOOK STORES,131.82414,29
`DEPARTMENT OF EDUCATION,"BOOKS, PERIODICALS AND NEWSPAPERS",275.00000,1
`DEPARTMENT OF EDUCATION,BUSINESS SERVICES NOT ELSEWHERE CLASSIFIED,235.88067,15


Selecting by mean_count_ratio


Agency_Name,Merchant_Category,Year_Month,max_amount_ratio,mean_amount_ratio,mean_count_ratio
`DEPARTMENT OF EDUCATION,ADVERTISING SERVICES,201307,1.48220403,0.82969836,0.60000000
`DEPARTMENT OF EDUCATION,ADVERTISING SERVICES,201308,0.07496765,0.07496765,0.20000000
`DEPARTMENT OF EDUCATION,ADVERTISING SERVICES,201401,2.43593727,2.43593727,0.20000000
`DEPARTMENT OF EDUCATION,AMERICAN AIRLINES,201307,1.72742510,1.44530199,0.13483146
`DEPARTMENT OF EDUCATION,AMERICAN AIRLINES,201308,1.46372704,1.07605034,0.05617978
`DEPARTMENT OF EDUCATION,AMERICAN AIRLINES,201312,1.09860666,-0.10453261,0.06741573
`DEPARTMENT OF EDUCATION,AMERICAN AIRLINES,201401,1.12781629,0.86284708,0.06741573
`DEPARTMENT OF EDUCATION,BEST WESTERN,201308,1.88458293,1.88458293,0.14285714
`DEPARTMENT OF EDUCATION,BEST WESTERN,201309,0.73830762,0.73830762,0.14285714
`DEPARTMENT OF EDUCATION,BEST WESTERN,201401,0.87542189,0.87542189,0.71428571


#### Feature group 1: Now create features for "max_amount_ratio"
The following step will create N features for max_amount_ratio

In [13]:
max_per_agency_category <- per_agency_category %>% 
    mutate(Year_Month = paste("Max",Year_Month,sep="_")) %>%
    select(-mean_amount_ratio, -mean_count_ratio)
    
head(max_per_agency_category)

Agency_Name,Merchant_Category,Year_Month,max_amount_ratio
`DEPARTMENT OF EDUCATION,ADVERTISING SERVICES,Max_201307,1.48220403
`DEPARTMENT OF EDUCATION,ADVERTISING SERVICES,Max_201308,0.07496765
`DEPARTMENT OF EDUCATION,ADVERTISING SERVICES,Max_201401,2.43593727
`DEPARTMENT OF EDUCATION,AMERICAN AIRLINES,Max_201307,1.72742510
`DEPARTMENT OF EDUCATION,AMERICAN AIRLINES,Max_201308,1.46372704
`DEPARTMENT OF EDUCATION,AMERICAN AIRLINES,Max_201312,1.09860666


In [15]:
# Use "dcast" in Library "reshape2" to organize the data so each row is a merchant category of an agent.
library(reshape2)
max_wide <- dcast(max_per_agency_category, Agency_Name + Merchant_Category ~ Year_Month)
max_wide=as.matrix(max_wide)
max_wide[is.na(max_wide)] <-0
wide=as.data.frame(max_wide)
head(max_wide)

Using max_amount_ratio as value column: use value.var to override.


Agency_Name,Merchant_Category,Max_201307,Max_201308,Max_201309,Max_201310,Max_201311,Max_201312,Max_201401,Max_201402,Max_201403,Max_201404,Max_201405,Max_201406
`DEPARTMENT OF EDUCATION,ADVERTISING SERVICES,1.482204e+00,7.496765e-02,0,0,0,0,2.435937e+00,0,0,0,0,0
`DEPARTMENT OF EDUCATION,AMERICAN AIRLINES,1.727425e+00,1.463727e+00,0,0,0,1.098607e+00,1.127816e+00,0,0,0,0,0
`DEPARTMENT OF EDUCATION,BEST WESTERN,0,1.884583e+00,7.383076e-01,0,0,0,8.754219e-01,0,0,0,0,0
`DEPARTMENT OF EDUCATION,BOOK STORES,2.911303e+00,5.317691e-01,3.179236e-01,4.301184964,0,6.221167e-01,0,0,0,0,0,0
`DEPARTMENT OF EDUCATION,"BOOKS, PERIODICALS AND NEWSPAPERS",0,0,0,1.000000000,0,0,0,0,0,0,0,0
`DEPARTMENT OF EDUCATION,BUSINESS SERVICES NOT ELSEWHERE CLASSIFIED,4.027460e-02,1.271830e+00,0,0,1.801759e+00,1.839701e+00,2.204505e+00,0,0,0,0,0


#### Feature group 2: mean_amount_ratio

In [17]:
mean_per_agency_category <- per_agency_category %>% 
    mutate(Year_Month = paste("Mean",Year_Month,sep="_")) %>%
    select(-max_amount_ratio, -mean_count_ratio)
    
head(mean_per_agency_category)

# Use "dcast" in Library "reshape2" to organize the data so each row is a merchant category of an agent.
mean_wide <- dcast(mean_per_agency_category, Agency_Name + Merchant_Category ~ Year_Month)
mean_wide=as.matrix(mean_wide)
mean_wide[is.na(mean_wide)] <-0
wide=as.data.frame(mean_wide)
head(mean_wide)

Agency_Name,Merchant_Category,Year_Month,mean_amount_ratio
`DEPARTMENT OF EDUCATION,ADVERTISING SERVICES,Mean_201307,0.82969836
`DEPARTMENT OF EDUCATION,ADVERTISING SERVICES,Mean_201308,0.07496765
`DEPARTMENT OF EDUCATION,ADVERTISING SERVICES,Mean_201401,2.43593727
`DEPARTMENT OF EDUCATION,AMERICAN AIRLINES,Mean_201307,1.44530199
`DEPARTMENT OF EDUCATION,AMERICAN AIRLINES,Mean_201308,1.07605034
`DEPARTMENT OF EDUCATION,AMERICAN AIRLINES,Mean_201312,-0.10453261


Using mean_amount_ratio as value column: use value.var to override.


Agency_Name,Merchant_Category,Mean_201307,Mean_201308,Mean_201309,Mean_201310,Mean_201311,Mean_201312,Mean_201401,Mean_201402,Mean_201403,Mean_201404,Mean_201405,Mean_201406
`DEPARTMENT OF EDUCATION,ADVERTISING SERVICES,8.296984e-01,7.496765e-02,0,0,0,0,2.435937e+00,0,0,0,0,0
`DEPARTMENT OF EDUCATION,AMERICAN AIRLINES,1.445302e+00,1.076050e+00,0,0,0,-0.1045326132,8.628471e-01,0,0,0,0,0
`DEPARTMENT OF EDUCATION,BEST WESTERN,0,1.884583e+00,7.383076e-01,0,0,0,8.754219e-01,0,0,0,0,0
`DEPARTMENT OF EDUCATION,BOOK STORES,9.944309e-01,5.317691e-01,3.179236e-01,1.403551754,0,0.4417627979,0,0,0,0,0,0
`DEPARTMENT OF EDUCATION,"BOOKS, PERIODICALS AND NEWSPAPERS",0,0,0,1.000000000,0,0,0,0,0,0,0,0
`DEPARTMENT OF EDUCATION,BUSINESS SERVICES NOT ELSEWHERE CLASSIFIED,4.027460e-02,5.065414e-01,0,0,1.8017585163,1.7571384966,9.489262e-01,0,0,0,0,0


### Hints:

1. Now you have two datasets "max_per_agency_category" and "mean_per_agency_category". Each dataset has N features. What do you do?  
2. You can create more feature groups by imitating the same procedure. Do you have an idea now?

### Next steps: 
You will prepare the modeling dataset for DBSCAN clustering. You are also encouraged to perform K-means to compare the results. The outliers will be identified by DBSCAN. Then you will use the principal component technique to reduce the features. The principal component is used only to visualize the results. 